### Unit Tests of FFSR Module Functions ###

In [ ]:
# To correct the following 2 cells, had to run this in SSH:
# conda install -c asmeurer readline
# delete __pycache__
# add "import readline" to the test files that were failing

### NOTE: the Unit_Tests notebook in the Tests directory is the final, correct version

In [22]:
################################################
# The expected error of test_covnames is a "TypeError"

from ffsr import *

import rpy2.robjects as ro
import pandas.rpy.common as com
from rpy2.robjects.packages import importr

import numpy as np
import pandas as pd

np.random.seed(1234)

X = np.random.multivariate_normal(np.zeros(15),np.eye(15),(100))
beta = np.array([0,0,5,6,0,0,4,0,0,0,5,0,0,0,0]).reshape(15,1) # signif betas: 3,4,7,11
Y = X.dot(beta)
Y2 = pd.DataFrame(Y)
X2 = pd.DataFrame(X)
X2.columns = ["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15"]

fwd_r = forward(X2,Y2)

cov_order(X2.columns.values)

cov_order(None)

TypeError: object of type 'NoneType' has no len()

In [28]:
################################################
# test_pvals.py should return "True"

from ffsr import *

import rpy2.robjects as ro
import pandas.rpy.common as com
from rpy2.robjects.packages import importr

import numpy as np
import pandas as pd

np.random.seed(1234)

X = np.random.multivariate_normal(np.zeros(15),np.eye(15),(100))
beta = np.array([0,0,5,6,0,0,4,0,0,0,5,0,0,0,0]).reshape(15,1) # signif betas: 3,4,7,11
Y = X.dot(beta)
Y2 = pd.DataFrame(Y)
X2 = pd.DataFrame(X)
X2.columns = ["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15"]

fwd_r = forward(X2,Y2)

codnames = cov_order(X2.columns.values)

0. <= np.all(pval_comp(X2.shape[1])) <= 1

True

In [5]:
################################################
#### Check errors for gamma_F

from ffsr import *

import numpy as np
import pandas as pd

np.random.seed(1234)

X = np.random.multivariate_normal(np.zeros(15),np.eye(15),(100))
beta = np.array([0,0,5,6,0,0,4,0,0,0,5,0,0,0,0]).reshape(15,1) # signif betas: 3,4,7,11
Y = X.dot(beta)
Y2 = pd.DataFrame(Y)
X2 = pd.DataFrame(X)
X2.columns = ["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15"]

fwd_r = forward(X2,Y2)
codnames = cov_order(X2.columns.values)
po = pval_comp(X2.shape[1])
pm = np.array([max(po[:(i+1)]) for i in range(len(po))])
s = np.array([len(np.where(pm<=pm[i])[0]) for i in range(len(pm))])

# gamma_F(None,15,s) # TypeError
# gamma_F(pm,10,s) # ValueError, ncov < len(pvals) --> gamma < 0
gamma_F(po[:10],10,s) # ValueError, max_size > length(pvals)

ValueError: Largest model size > len(pvals) --> gamma < 0

In [16]:
################################################
#### Check errors for alpha_F

from ffsr import *

import numpy as np
import pandas as pd

s = np.arange(1,15)

# alpha_F(None,15,s) # TypeError
alpha_F(0.05,None,s) # TypeError
# alpha_F(0.05,10,s) # ValueError
# alpha_F(1,15,s) # ValueError
# alpha_F(-0.1,15,s) # ValueError

s = np.arange(1,5)
n = 5
a = 0.5 * (1 + s) / (n - s)
af = alpha_F(0.5,n,np.arange(1,n))
np.allclose(a,af)

TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'

In [11]:
################################################
#### Check errors for alpha_F_g

from ffsr import alpha_F_g

import numpy as np
import pandas as pd

np.random.seed(1234)

g = np.random.random(15)

# alpha_F_g(1,g,15) # ValueError
# alpha_F_g(-0.1,g,15) # ValueError
# alpha_F_g(None,g,15) # ValueError
# alpha_F_g(0.05,None,15) # ValueError
# alpha_F_g(0.05,g,None) # TypeError

g2 = np.array([0.01,0.02,0.03])

# alpha_F_g(np.array([0.05,1]),g,15) # ValueError
# alpha_F_g(np.array([-0.05,0.05]),g,15) # ValueError
# alpha_F_g(np.array([-0.05,0.05,1]),g,15) # ValueError
# alpha_F_g(g2,None,15) # ValueError
# alpha_F_g(g2,g,None) # TypeError

g3 = np.array([0.1,0.5])
n = 5
g0 = 0.25
S = min(np.where(g3>g0)[0])
a = g0 * (1. + s) / (n - s)
afg = alpha_F_g(g0,g3,n)

0.125 0.125


True

In [5]:
################################################
#### Check errors for alpha_F_g

from ffsr import beta_est

import numpy as np
import pandas as pd

np.random.seed(1234)

g = np.random.random(15)

X = np.random.multivariate_normal(np.zeros(15),np.eye(15),(100))
beta = np.array([0,0,5,6,0,0,4,0,0,0,5,0,0,0,0]).reshape(15,1) # signif betas: 3,4,7,11
Y = X.dot(beta)
Y2 = pd.DataFrame(Y)
X2 = pd.DataFrame(X)
X2.columns = ["V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15"]

# beta_est(None,Y2,0.05,g,X2.columns.values) # AttributeError
# beta_est(X2,None,0.05,g,X2.columns.values) # TypeError
# beta_est(X2,Y2,None,g,X2.columns.values) # ValueError
# beta_est(X2,Y2,0.05,None,X2.columns.values) # ValueError
# beta_est(X2,Y2,0.05,g,None) # TypeError

from numpy.testing import assert_raises

assert_raises(AttributeError,beta_est,None,Y2,0.05,g,X2.coumns.values)

AttributeError: 'DataFrame' object has no attribute 'coumns'